In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque
%matplotlib inline

### Camera calibration & distortion correction.

In [2]:
def calibrate_camera():
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return objpoints, imgpoints

In [3]:
def img_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return cv2.undistort(img, mtx, dist, None, mtx)

### Color transforms & gradients.

In [4]:
# Function that takes an image, gradient orientation,
# and threshold min / max values. This function applies Sobel x or y,
# then takes an absolute value and applies a threshold.
def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

In [5]:
# Function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

In [6]:
# Function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

In [7]:
# Define a function that thresholds the S-channel of HLS
def hls_select_channel(img, channel, thresh = (0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if channel == 'l':
        channel = hls[:,:,1]
    else:
        channel = hls[:,:,2]
    binary_output = np.zeros_like(channel)
    binary_output[(channel > thresh[0]) & (channel <= thresh[1])] = 1
    return binary_output

In [8]:
# Function that applies binarization
def binarize(img):
    ksize = 3
    
    gradx = abs_sobel_thresh(img, orient='x', thresh=(20, 255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25, 100))
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(img, sobel_kernel = ksize, thresh=(0.7, 1.3))
    hls_s_binary = hls_select_channel(img, 's', thresh=(120, 255))
    hls_l_binary = hls_select_channel(img, 'l', thresh=(50, 255))

    combined = np.zeros_like(dir_binary)
    #combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | ((hls_s_binary == 1) & (hls_l_binary == 1))] = 1
    combined[((gradx == 1) & (grady == 1)) | ((hls_s_binary == 1) & (hls_l_binary == 1))] = 1

    return combined

### Perspective transform.

In [9]:
def warp(img):
    src = np.float32([[190, 720], [582, 457], [701, 457], [1145, 720]])
    offset = [150,0]
    dst = np.float32([src[0] + offset, np.array([src[0, 0], 0]) + offset, np.array([src[3, 0], 0]) - offset, src[3] - offset])
    
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags = cv2.INTER_LINEAR)
        
    return warped, M

### Identiffying lane-line pixels and fittig their positions with a polynomial.

In [10]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, n = 5):
        # length of queue to store data
        self.n = n
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = deque([], maxlen = n) 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        # fit coefficients of the last n iterations
        self.recent_fit_coeffs = deque([], maxlen = n)    
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units (meters)
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #y values for line fit (ploty)
        self.fit_y = None
        
    def calculate_and_set_curvature(self):
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        
        y_eval = np.max(self.fit_y)

        # Fit new polynomials to x,y in world space
        fit_cr = np.polyfit(self.fit_y * ym_per_pix, self.bestx * xm_per_pix, 2)
        # Calculate the new radius of curvature
        self.radius_of_curvature = ((1 + (2 * fit_cr[0] * y_eval * ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2 * fit_cr[0])
        
    def update(self, detected, current_fit, fitx, x, y, fit_y):
        self.detected = detected
        self.recent_xfitted.appendleft(fitx)
        self.bestx = np.mean(self.recent_xfitted, axis = 0)
        self.current_fit = current_fit
        self.allx = x  
        self.ally = y
        self.fit_y = fit_y
        self.calculate_and_set_curvature()

In [11]:
def detect_and_update_lines(binary_warped):
    
    global left_line
    global right_line
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Set the width of the windows +/- margin
    margin = 100
        
    # Check if lines were last detected; if not detect lines using histogram
    if left_line.detected == False | right_line.detected == False:

        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0] / 2:, :], axis = 0)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0] / 2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0] / nwindows)
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_line_inds = []
        right_line_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window + 1) * window_height
            win_y_high = binary_warped.shape[0] - window * window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_line_inds.append(good_left_inds)
            right_line_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_line_inds = np.concatenate(left_line_inds)
        right_line_inds = np.concatenate(right_line_inds)
        
    else:
        # We have a new warped binary image from the next frame of video
        left_line_inds = ((nonzerox > (left_line.current_fit[0] * (nonzeroy**2) + left_line.current_fit[1] * nonzeroy + left_line.current_fit[2] - margin)) & (nonzerox < (left_line.current_fit[0] * (nonzeroy**2) + left_line.current_fit[1] * nonzeroy + left_line.current_fit[2] + margin))) 
        right_line_inds = ((nonzerox > (right_line.current_fit[0] * (nonzeroy**2) + right_line.current_fit[1] * nonzeroy + right_line.current_fit[2] - margin)) & (nonzerox < (right_line.current_fit[0] * (nonzeroy**2) + right_line.current_fit[1] * nonzeroy + right_line.current_fit[2] + margin)))  
        
    # Extract left and right line pixel positions
    leftx = nonzerox[left_line_inds]
    lefty = nonzeroy[left_line_inds] 
    rightx = nonzerox[right_line_inds]
    righty = nonzeroy[right_line_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

    left_line.update(True, left_fit, left_fitx, leftx, lefty, ploty)
    right_line.update(True, right_fit, right_fitx, rightx, righty, ploty)

### Radius of curvature of the lane and the position of the vehicle with respect to center.

In [12]:
def calculate_distance_from_center(img):

    global left_line
    global right_line
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    left_fitx = left_line.current_fit
    right_fitx = right_line.current_fit
    left_b_x = left_fitx[left_fitx.size - 1]
    right_b_x = right_fitx[right_fitx.size - 1]
    expected_mid = (right_b_x - left_b_x) / 2 + left_b_x
    missed_mid = round((640 - expected_mid) * xm_per_pix * 100, 2)
    
    # Calculate distance from center and list differently based on left or right
    if missed_mid >= 0:
        center_text = "{} cm left of center".format(missed_mid)
    else:
        center_text = "{} cm right of center".format(-missed_mid)
        
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, center_text, (30,60), font, 1, (255,165,0), 2, cv2.LINE_AA)

In [13]:
def project_lane_curvature(img):
    global left_line
    global right_line
    
    lane_radius_of_curvature = round(np.mean([left_line.radius_of_curvature, right_line.radius_of_curvature]), 0)
    rad_text = "Radius of Curvature = {}(m)".format(lane_radius_of_curvature)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, rad_text, (30,100), font, 1, (255,165,0), 2, cv2.LINE_AA)

### Example result plotted back down onto the road.

In [14]:
def process_image(img):
    global left_line
    global right_line
    
    undist = img_undistort(img, objpoints, imgpoints)
    binary  = binarize(undist)
    warped = warp(binary)
    binary_warped = warped[0]
    
    detect_and_update_lines(binary_warped)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx = left_line.bestx
    right_fitx = right_line.bestx
    
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    perspective_transform_Minv = np.linalg.inv(warped[1])
    newwarp = cv2.warpPerspective(color_warp, perspective_transform_Minv, (img.shape[1], img.shape[0])) 

    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    calculate_distance_from_center(result)
    project_lane_curvature(result)
    
    return result

In [15]:
calibration_result = calibrate_camera()
objpoints = calibration_result[0]
imgpoints = calibration_result[1]

left_line = Line()
right_line = Line()

output = './result_project_video.mp4'
clip = VideoFileClip("project_video.mp4")
out_clip = clip.fl_image(process_image) 
%time out_clip.write_videofile(output, audio = False)

[MoviePy] >>>> Building video ./result_project_video.mp4
[MoviePy] Writing video ./result_project_video.mp4


100%|█████████▉| 1260/1261 [15:16<00:00,  1.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./result_project_video.mp4 

CPU times: user 17min 38s, sys: 1min 50s, total: 19min 29s
Wall time: 15min 17s
